# Summary

This notebook outlines the steps used to extract clinical data from the NCI Clinical Trial Data Commons and Genomic Data commons related to the Biobank, CPTAC and TCGA projects which have imaging data in The Cancer Imaging Archive (TCIA).  The resulting data will be merged with TCIA "Community" proposals that have clinical data to allow cohort building across as many TCIA datasets as possible.

# Setup

In [ ]:
# for CTDC
!pip install gql

# for TCIA
!pip install tcia_utils

In [21]:
import pandas as pd

# for TCIA
from tcia_utils import nbia
from tcia_utils import wordpress
from tcia_utils import pathdb

# for GDC
import json
import requests
import io

# for CTDC
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Get inventory of Radiology and Pathology data from TCIA

Get list of subjects that have radiology images.

In [4]:
nbia.getToken()

Enter User: 


 kirbyju
Enter Password:  ········


200

In [5]:
radiology = nbia.getPatient(format = 'df')

Get list of subjects that have pathology images.  

TODO: Not clear whether it's better to rely on Eaglescope or go directly to pathDB in the long term.  Instructions are here for both for the time being, but it seems like pathDB is best since it's more likely to be fully up to date.

In [4]:
# eaglescope approach
# Define the URL of the CSV file feeding Eaglescope
url = "https://pathdb.cancerimagingarchive.net/system/files/collectionmetadata/202401/cohort_builder_v1_01-16-2024.csv"

try:
  # Download and read the CSV data
  pathology = pd.read_csv(url)

  # Drop unwanted columns
 # df = df[['collection', 'collection_doi', 'patient_id', 'slide_id']]

  # Create a new column with the base URL and slide ID
  pathology['viewer_url'] = "https://pathdb.cancerimagingarchive.net/caMicroscope/apps/mini/viewer.html?mode=pathdb&slideid=" + pathology['slide_id'].astype(str)

  # Prefix collection_doi with https://doi.org/ and make it lowercase
  pathology['collection_doi'] = "https://doi.org/" + pathology['collection_doi'].astype(str).str.lower()

  print("Successfully processed data into DataFrame.")
except Exception as e:
  print(f"An error occurred: {e}")

Successfully processed data into DataFrame.


/var/folders/19/4p5m1b9j5k9f7qybvc64fbb454krph/T/ipykernel_60678/153889990.py:6: DtypeWarning: Columns (3,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  pathology = pd.read_csv(url)


In [6]:
# get patient/image metadata (only focused on NIH programs for now - Note: we don't host path for TCGA)
# if community pathology collections are added later we'll need to create a category for that here
cptacPath = pathdb.getImages(query = "cptac", format = "df")
biobankPath = pathdb.getImages(query = "cmb", format = "df")
upenngbmPath = pathdb.getImages(query = "upenn-gbm", format = "df")

In [7]:
# concatenate dataframes
pathology = pd.concat([cptacPath, biobankPath, upenngbmPath], ignore_index=True)

# rename to match expected header
pathology.rename(columns={'subjectId': 'patient_id', 'collectionName': 'collection'}, inplace=True)

pathology

,collection,collectionId,patient_id,imageId,imageHeight,imagedWidth,physicalPixelSizeX,physicalPixelSizeY,imageUrl,created,changed
0,CPTAC_AML,520,C3L-00452,C3L-00452-41,85678,92344,0.24670,0.246700,http://pathdb.cancerimagingarchive.net/system/...,2022-01-31 15-29-13,2022-01-31 15-30-19
1,CPTAC_AML,520,C3L-00453,C3L-00453-41,81273,80920,0.24670,0.246700,http://pathdb.cancerimagingarchive.net/system/...,2022-01-31 15-29-13,2022-01-31 15-31-19
2,CPTAC_AML,520,C3L-00455,C3L-00455-41,91616,97104,0.24670,0.246700,http://pathdb.cancerimagingarchive.net/system/...,2022-01-31 15-29-13,2022-01-31 15-31-19
3,CPTAC_AML,520,C3L-00457,C3L-00457-43,91134,94248,0.24670,0.246700,http://pathdb.cancerimagingarchive.net/system/...,2022-01-31 15-29-13,2022-01-31 15-31-19
4,CPTAC_AML,520,C3L-00458,C3L-00458-41,96263,96152,0.24670,0.246700,http://pathdb.cancerimagingarchive.net/system/...,2022-01-31 15-29-14,2022-01-31 15-31-19
...,...,...,...,...,...,...,...,...,...,...,...
8497,UPENN-GBM,543,C827913,7316UP-2058,87552,96000,0.23016,0.230165,http://pathdb.cancerimagingarchive.net/system/...,2022-10-24 13-20-44,2022-10-24 14-02-36
8498,UPENN-GBM,543,C827913,7316UP-2333,89856,130560,0.23016,0.230165,http://pathdb.cancerimagingarchive.net/system/...,2022-10-24 13-20-45,2022-10-24 14-02-36
8499,UPENN-GBM,543,C827913,7316UP-2527,105984,80640,0.23016,0.230165,http://pathdb.cancerimagingarchive.net/system/...,2022-10-24 13-20-46,2022-10-24 14-02-36
8500,UPENN-GBM,543,C846363,7316UP-2100,99072,138240,0.23016,0.230165,http://pathdb.cancerimagingarchive.net/system/...,2022-10-24 13-20-44,2022-10-24 14-02-36


If using pathDB we need to fix some collection values:

In [8]:
# Replace underscores with hyphens in the 'collection' column
pathology['collection'] = pathology['collection'].str.replace('_', '-')

# Fix data entry issues in PathDB that resulted in collection names that don't align with other TCIA systems
pathology['collection'] = pathology['collection'].replace('CPTAC-Glio-CODEX', 'CPTAC-GBM')
pathology['collection'] = pathology['collection'].replace('CPTAC-non-CCRCC', 'CPTAC-CCRCC')

# Extract unique values from the 'collection' column 
unique_collections = pathology['collection'].unique().tolist()

print(unique_collections)


['CPTAC-AML', 'CPTAC-BRCA', 'CPTAC-CCRCC', 'CPTAC-CM', 'CPTAC-COAD', 'CPTAC-GBM', 'CPTAC-HNSCC', 'CPTAC-LSCC', 'CPTAC-LUAD', 'CPTAC-OV', 'CPTAC-PDA', 'CPTAC-SAR', 'CPTAC-UCEC', 'CMB-AML', 'CMB-BRCA', 'CMB-CRC', 'CMB-GEC', 'CMB-LCA', 'CMB-MEL', 'CMB-MML', 'CMB-OV', 'CMB-PCA', 'UPENN-GBM']


Create function to add status of available images to clinical dataframes and generate rad/path patient lists.

In [9]:
def add_image_availability(df, pathology_patients, radiology_patients):
    """Adds a new column 'Available Images' to the DataFrame indicating image availability.

    Args:
        df: The DataFrame to modify.
        pathology_patients: A list of unique patient IDs from the pathology data.
        radiology_patients: A list of unique patient IDs from the radiology data.

    Returns:
        The modified DataFrame.
    """
    # Ensure we are working on a copy of the DataFrame
    df = df.copy()
    
    df['Available Images'] = df['Case ID'].apply(
        lambda x: 'Radiology; Pathology' if x in pathology_patients and x in radiology_patients 
        else 'Pathology' if x in pathology_patients 
        else 'Radiology' if x in radiology_patients 
        else 'None'
    )
    return df

# Create lists of rad/path patient IDs
pathology_patients = pathology['patient_id'].unique().tolist()
radiology_patients = radiology['PatientId'].unique().tolist()

# Get Biobank clinical data from Clinical Trials Data Commons

In [45]:
# Define the transport
transport = RequestsHTTPTransport(
    url="https://clinical.datacommons.cancer.gov/v1/graphql/",
    verify=True,
    retries=3,
)

# Create a client instance, disabling schema fetching
client = Client(transport=transport, fetch_schema_from_transport=False)

# Define the query and variables
query = gql("""
  query participantOverview($subject_id: [String], $ctep_disease_term: [String], $stage_of_disease: [String], $tumor_grade: [String], $sex: [String], $reported_gender: [String], $race: [String], $ethnicity: [String], $carcinogen_exposure: [String], $targeted_therapy: [String], $anatomical_collection_site: [String], $tissue_category: [String], $assessment_timepoint: [String], $data_file_type: [String], $data_file_format: [String], $first: Int, $offset: Int, $order_by: String, $sort_direction: String) {
    participantOverview(subject_id: $subject_id, ctep_disease_term: $ctep_disease_term, stage_of_disease: $stage_of_disease, tumor_grade: $tumor_grade, sex: $sex, reported_gender: $reported_gender, race: $race, ethnicity: $ethnicity, carcinogen_exposure: $carcinogen_exposure, targeted_therapy: $targeted_therapy, anatomical_collection_site: $anatomical_collection_site, tissue_category: $tissue_category, assessment_timepoint: $assessment_timepoint, data_file_type: $data_file_type, data_file_format: $data_file_format, first: $first, offset: $offset, order_by: $order_by, sort_direction: $sort_direction) {
      subject_id
      ctep_disease_term
      stage_of_disease
      tumor_grade
      age_at_enrollment
      sex
      reported_gender
      race
      ethnicity
      carcinogen_exposure
      targeted_therapy
      data_file_uuid
      __typename
    }
  }
""")

variables = {
    "first": 10000
}

# Execute the query
result = client.execute(query, variable_values=variables)

# Execute the query
result = client.execute(query, variable_values=variables)

# Extract the data from the result
data = result.get("participantOverview", [])

# Convert the data into a pandas DataFrame
biobank = pd.DataFrame(data)

biobank = biobank.rename(columns={'subject_id': 'Case ID'})

#biobank.to_excel('ctdc.xlsx')
biobank

,Case ID,ctep_disease_term,stage_of_disease,tumor_grade,age_at_enrollment,sex,reported_gender,race,ethnicity,carcinogen_exposure,targeted_therapy,data_file_uuid,__typename
0,MSB-00089,Plasma Cell Myeloma,,,67.0,Female,,Black or African American,Not Hispanic or Latino,No,"[Bortezomib, Lenalidomide]",[],ParticipantOverview
1,MSB-00140,Plasma Cell Myeloma,,,65.0,Female,,White,Not Hispanic or Latino,No,"[Bortezomib, Lenalidomide]","[dg.4DFC/CC00EEF6-1730-40B3-9FA6-3D8FCD39EB83,...",ParticipantOverview
2,MSB-00205,Melanoma,,Grade cannot be assessed,63.0,Male,Male,White,Not Hispanic or Latino,No,[],[],ParticipantOverview
3,MSB-00241,Colorectal Carcinoma,,,59.0,Female,,White,Not Hispanic or Latino,Unknown,[],[dg.4DFC/7E4EE0AD-DFCC-4AEA-BA47-9C94E6A6E116],ParticipantOverview
4,MSB-00263,Melanoma,,Grade cannot be assessed,56.0,Male,Male,White,Not Hispanic or Latino,Unknown,[],"[dg.4DFC/640C5DFC-B8A1-44AD-B950-467ABC123F45,...",ParticipantOverview
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,MSB-09886,Colorectal Carcinoma,Stage IVB,Moderately Differentiated,67.0,Male,,White,Not Hispanic or Latino,Unknown,[Bevacizumab],[],ParticipantOverview
244,MSB-09897,Colorectal Carcinoma,Stage IVA,Moderately Differentiated,59.0,Male,,White,Hispanic or Latino,Unknown,[Bevacizumab],[],ParticipantOverview
245,MSB-09977,Small Cell Lung Carcinoma,Stage IV,Grade cannot be assessed,79.0,Male,,White,Not Hispanic or Latino,Unknown,[],"[dg.4DFC/A94E8653-C843-4D2F-9860-67E1BAFAD3F0,...",ParticipantOverview
246,MSB-09991,Colorectal Carcinoma,Stage IV,Moderately Differentiated,66.0,Male,,White,Not Hispanic or Latino,No,"[Bevacizumab, Cetuximab]","[dg.4DFC/D69D7DA7-D19F-4A6A-9523-3026F6B03A22,...",ParticipantOverview


CTDC doesn't include the collection name in their API results so we need to add it

In [10]:
def add_collection_from_other_dfs(df, pathology, radiology):
    """Adds a 'Collection' column to the DataFrame based on matches in pathology and radiology DataFrames.

    Args:
        df: The input DataFrame.
        pathology: The pathology DataFrame.
        radiology: The radiology DataFrame.

    Returns:
        The modified DataFrame.
    """
    # Create a dictionary mapping patient IDs to collections from pathology
    pathology_map = pathology.set_index('patient_id')['collection'].to_dict()

    # Remove entries where the 'collection' value is 'CPTAC-Glioblastoma-CODEX' as this is an analysis result with overlapping subjects in multiple collections 
    # No longer necessary to do this now that we're converting these subjects to CPTAC-GBM in earlier cleanup?
    #pathology_map = {k: v for k, v in pathology_map.items() if v != 'CPTAC-Glioblastoma-CODEX'}

    # Create a dictionary mapping patient IDs to collections from radiology
    radiology_map = radiology.set_index('PatientId')['Collection'].to_dict()

    # Add the 'Collection' column to the biobank DataFrame
    df['Project Short Name'] = df['Case ID'].map(radiology_map).fillna(df['Case ID'].map(pathology_map))

    return df




In [46]:
# Add the 'Project Short Name' column to the biobank DataFrame
biobank_with_project_name = add_collection_from_other_dfs(biobank, pathology, radiology)

biobank_with_project_name

,Case ID,ctep_disease_term,stage_of_disease,tumor_grade,age_at_enrollment,sex,reported_gender,race,ethnicity,carcinogen_exposure,targeted_therapy,data_file_uuid,__typename,Project Short Name
0,MSB-00089,Plasma Cell Myeloma,,,67.0,Female,,Black or African American,Not Hispanic or Latino,No,"[Bortezomib, Lenalidomide]",[],ParticipantOverview,CMB-MML
1,MSB-00140,Plasma Cell Myeloma,,,65.0,Female,,White,Not Hispanic or Latino,No,"[Bortezomib, Lenalidomide]","[dg.4DFC/CC00EEF6-1730-40B3-9FA6-3D8FCD39EB83,...",ParticipantOverview,CMB-MML
2,MSB-00205,Melanoma,,Grade cannot be assessed,63.0,Male,Male,White,Not Hispanic or Latino,No,[],[],ParticipantOverview,CMB-MEL
3,MSB-00241,Colorectal Carcinoma,,,59.0,Female,,White,Not Hispanic or Latino,Unknown,[],[dg.4DFC/7E4EE0AD-DFCC-4AEA-BA47-9C94E6A6E116],ParticipantOverview,CMB-CRC
4,MSB-00263,Melanoma,,Grade cannot be assessed,56.0,Male,Male,White,Not Hispanic or Latino,Unknown,[],"[dg.4DFC/640C5DFC-B8A1-44AD-B950-467ABC123F45,...",ParticipantOverview,CMB-MEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,MSB-09886,Colorectal Carcinoma,Stage IVB,Moderately Differentiated,67.0,Male,,White,Not Hispanic or Latino,Unknown,[Bevacizumab],[],ParticipantOverview,CMB-CRC
244,MSB-09897,Colorectal Carcinoma,Stage IVA,Moderately Differentiated,59.0,Male,,White,Hispanic or Latino,Unknown,[Bevacizumab],[],ParticipantOverview,CMB-CRC
245,MSB-09977,Small Cell Lung Carcinoma,Stage IV,Grade cannot be assessed,79.0,Male,,White,Not Hispanic or Latino,Unknown,[],"[dg.4DFC/A94E8653-C843-4D2F-9860-67E1BAFAD3F0,...",ParticipantOverview,CMB-LCA
246,MSB-09991,Colorectal Carcinoma,Stage IV,Moderately Differentiated,66.0,Male,,White,Not Hispanic or Latino,No,"[Bevacizumab, Cetuximab]","[dg.4DFC/D69D7DA7-D19F-4A6A-9523-3026F6B03A22,...",ParticipantOverview,CMB-CRC


In [47]:
# Add the 'available_images' column 
biobank_with_project_and_image_availability = add_image_availability(biobank_with_project_name, pathology_patients, radiology_patients)

biobank_with_project_and_image_availability

,Case ID,ctep_disease_term,stage_of_disease,tumor_grade,age_at_enrollment,sex,reported_gender,race,ethnicity,carcinogen_exposure,targeted_therapy,data_file_uuid,__typename,Project Short Name,Available Images
0,MSB-00089,Plasma Cell Myeloma,,,67.0,Female,,Black or African American,Not Hispanic or Latino,No,"[Bortezomib, Lenalidomide]",[],ParticipantOverview,CMB-MML,Pathology
1,MSB-00140,Plasma Cell Myeloma,,,65.0,Female,,White,Not Hispanic or Latino,No,"[Bortezomib, Lenalidomide]","[dg.4DFC/CC00EEF6-1730-40B3-9FA6-3D8FCD39EB83,...",ParticipantOverview,CMB-MML,Radiology; Pathology
2,MSB-00205,Melanoma,,Grade cannot be assessed,63.0,Male,Male,White,Not Hispanic or Latino,No,[],[],ParticipantOverview,CMB-MEL,Pathology
3,MSB-00241,Colorectal Carcinoma,,,59.0,Female,,White,Not Hispanic or Latino,Unknown,[],[dg.4DFC/7E4EE0AD-DFCC-4AEA-BA47-9C94E6A6E116],ParticipantOverview,CMB-CRC,Pathology
4,MSB-00263,Melanoma,,Grade cannot be assessed,56.0,Male,Male,White,Not Hispanic or Latino,Unknown,[],"[dg.4DFC/640C5DFC-B8A1-44AD-B950-467ABC123F45,...",ParticipantOverview,CMB-MEL,Radiology; Pathology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,MSB-09886,Colorectal Carcinoma,Stage IVB,Moderately Differentiated,67.0,Male,,White,Not Hispanic or Latino,Unknown,[Bevacizumab],[],ParticipantOverview,CMB-CRC,Radiology; Pathology
244,MSB-09897,Colorectal Carcinoma,Stage IVA,Moderately Differentiated,59.0,Male,,White,Hispanic or Latino,Unknown,[Bevacizumab],[],ParticipantOverview,CMB-CRC,Pathology
245,MSB-09977,Small Cell Lung Carcinoma,Stage IV,Grade cannot be assessed,79.0,Male,,White,Not Hispanic or Latino,Unknown,[],"[dg.4DFC/A94E8653-C843-4D2F-9860-67E1BAFAD3F0,...",ParticipantOverview,CMB-LCA,Radiology; Pathology
246,MSB-09991,Colorectal Carcinoma,Stage IV,Moderately Differentiated,66.0,Male,,White,Not Hispanic or Latino,No,"[Bevacizumab, Cetuximab]","[dg.4DFC/D69D7DA7-D19F-4A6A-9523-3026F6B03A22,...",ParticipantOverview,CMB-CRC,Radiology; Pathology


# apply data standardization steps to align with output from https://tcia-clinical-validator.streamlit.app/

* Create a new dataframe called 'master_clinical' from 'biobank_with_project_and_image_availability' dataframe
* Include and rename 'ctep_disease_term' -> 'Primary Diagnosis' column. 
* Include and rename 'age_at_enrollment' -> 'Age at Enrollment' column.
* Include 'Race' and 'Ethnicity' columns, but capitalize them (they're called 'race' and 'ethnicity' in the biobank dataframe).  
* Include and rename 'sex' -> 'Sex at Birth' column.
* Create 'Age UOM' column and populate all rows with 'Year' as the value
* Convert the word 'or' in all values that are found in the Race and Ethnicity columns to be lower case
* Include 'Project Short Name' and 'available_images' 


In [48]:
# check for Case ID values that we didn't find a corresponding imaging Project Short Name
# CTDC team thinks they may have just never collected images for these subjects
null_values = biobank_with_project_name[biobank_with_project_name['Project Short Name'].isnull()]['Case ID'].tolist()

null_values

['MSB-01363', 'MSB-02746', 'MSB-03272', 'MSB-07370']

In [49]:
# Extract unique values from the 'collection' column 
unique_diseases = biobank_with_project_and_image_availability['ctep_disease_term'].unique().tolist()

print(unique_diseases)

['Plasma Cell Myeloma', 'Melanoma', 'Colorectal Carcinoma', 'Non-Small Cell Lung Carcinoma', 'Small Cell Lung Carcinoma', 'Adenocarcinoma of the Gastroesophageal Junction', 'Acute Myeloid Leukemia Not Otherwise Specified', 'Prostate Carcinoma']


In [50]:
biobank_clinical = biobank_with_project_and_image_availability[['Project Short Name', 'Case ID', 'Available Images', 'ctep_disease_term', 'race', 'ethnicity', 'sex', 'age_at_enrollment']].copy()

# drop rows where 'subject_id' is null (not clear why 4 subjects show up in CTDC but not in TCIA -- CTDC team investigating)
biobank_clinical = biobank_clinical.dropna(subset=['Project Short Name'])

# Rename columns
biobank_clinical.rename(columns={'ctep_disease_term': 'Primary Diagnosis',
                               'age_at_enrollment': 'Age at Enrollment',
                               'sex': 'Sex at Birth',
                               'race': 'Race',
                               'ethnicity':'Ethnicity'}, inplace=True)


# Convert 'or' to lowercase in Race and Ethnicity
biobank_clinical['Race'] = biobank_clinical['Race'].str.replace('Or', 'or', case=False)
biobank_clinical['Race'] = biobank_clinical['Race'].str.replace('or other', 'or Other', case=False)
biobank_clinical['Ethnicity'] = biobank_clinical['Ethnicity'].str.replace('Or', 'or', case=False)

# Create 'Age UOM' column
biobank_clinical['Age UOM'] = 'Year'

In [51]:
# Extract unique values from the specified columns to ensure data formatting
unique_race = biobank_clinical['Race'].unique().tolist()
unique_ethnicity = biobank_clinical['Ethnicity'].unique().tolist()
unique_sex_at_birth = biobank_clinical['Sex at Birth'].unique().tolist()

print("Unique Race values:", unique_race)
print("Unique Ethnicity values:", unique_ethnicity)
print("Unique Sex at Birth values:", unique_sex_at_birth)

Unique Race values: ['Black or African American', 'White', 'Not Reported', 'Asian', 'Unknown', 'American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander']
Unique Ethnicity values: ['Not Hispanic or Latino', 'Hispanic or Latino', 'Not Reported', 'Unknown']
Unique Sex at Birth values: ['Female', 'Male']


In [52]:
# Calculate the minimum and maximum ages in the 'Age at Enrollment' column
min_age = biobank_clinical['Age at Enrollment'].min()
max_age = biobank_clinical['Age at Enrollment'].max()

print(f"The range of ages in 'Age at Enrollment' is from {min_age} to {max_age}.")


The range of ages in 'Age at Enrollment' is from 28.0 to 88.0.


In [53]:
# Double check for subjects without imaging using 'Available Images' (i.e. 'None')
None_images = biobank_clinical[biobank_clinical['Available Images'] == 'None']

# Display the filtered rows
print(None_images)


Empty DataFrame
Columns: [Project Short Name, Case ID, Available Images, Primary Diagnosis, Race, Ethnicity, Sex at Birth, Age at Enrollment, Age UOM]
Index: []


In [54]:
biobank_clinical

,Project Short Name,Case ID,Available Images,Primary Diagnosis,Race,Ethnicity,Sex at Birth,Age at Enrollment,Age UOM
0,CMB-MML,MSB-00089,Pathology,Plasma Cell Myeloma,Black or African American,Not Hispanic or Latino,Female,67.0,Year
1,CMB-MML,MSB-00140,Radiology; Pathology,Plasma Cell Myeloma,White,Not Hispanic or Latino,Female,65.0,Year
2,CMB-MEL,MSB-00205,Pathology,Melanoma,White,Not Hispanic or Latino,Male,63.0,Year
3,CMB-CRC,MSB-00241,Pathology,Colorectal Carcinoma,White,Not Hispanic or Latino,Female,59.0,Year
4,CMB-MEL,MSB-00263,Radiology; Pathology,Melanoma,White,Not Hispanic or Latino,Male,56.0,Year
...,...,...,...,...,...,...,...,...,...
243,CMB-CRC,MSB-09886,Radiology; Pathology,Colorectal Carcinoma,White,Not Hispanic or Latino,Male,67.0,Year
244,CMB-CRC,MSB-09897,Pathology,Colorectal Carcinoma,White,Hispanic or Latino,Male,59.0,Year
245,CMB-LCA,MSB-09977,Radiology; Pathology,Small Cell Lung Carcinoma,White,Not Hispanic or Latino,Male,79.0,Year
246,CMB-CRC,MSB-09991,Radiology; Pathology,Colorectal Carcinoma,White,Not Hispanic or Latino,Male,66.0,Year


In [55]:
biobank_clinical.to_excel('biobank_clinical_final.xlsx')

Biobank data seems good to go!

# Get CPTAC from PDC

GDC doesn't seem to have CPTAC-SAR, CPTAC-CM, and CPTAC-AML so we'll try PDC for CPTAC

In [56]:
file_path = '/Users/kirbyju/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/misc downloads/PDC_clinical_data_manifests_11122024_000423/PDC_clinical_manifest_11122024_000423.tsv' 
pdc_cptac = pd.read_csv(file_path, delimiter='\t')

In [57]:
cptac_clinical = pdc_cptac[['Case Submitter ID', 'Primary Diagnosis', 'Primary Site', 'Race', 'Ethnicity', 'Gender', 'Age at Diagnosis']].copy()

In [58]:
# Rename columns
cptac_clinical.rename(columns={'Case Submitter ID': 'Case ID',
                               'Gender': 'Sex at Birth'}, inplace=True)

# Create 'Age UOM' column
# note: PDC has some Ovarian subject values that look like they may be encoded as years, 
# but they do not seem to be subjects TCIA hosts
cptac_clinical['Age UOM'] = 'Day'

In [59]:
# Add the 'Project Short Name' column to the biobank DataFrame
cptac_with_projects = add_collection_from_other_dfs(cptac_clinical, pathology, radiology)

# drop rows where 'project short name' is null (so no match to tcia)
cptac_with_projects = cptac_clinical.dropna(subset=['Project Short Name'])

cptac_with_projects

,Case ID,Primary Diagnosis,Primary Site,Race,Ethnicity,Sex at Birth,Age at Diagnosis,Age UOM,Project Short Name
0,C3N-00721,"Renal cell carcinoma, NOS",Kidney,white,not hispanic or latino,male,,Day,CPTAC-CCRCC
1,C3L-02556,"Renal cell carcinoma, NOS",Kidney,white,not hispanic or latino,male,31315,Day,CPTAC-CCRCC
2,C3N-01657,"Renal cell carcinoma, NOS",Kidney,white,not reported,male,27232,Day,CPTAC-CCRCC
3,C3L-00970,"Renal cell carcinoma, NOS",Kidney,other,not reported,male,20187,Day,CPTAC-CCRCC
4,C3L-00991,"Renal cell carcinoma, NOS",Kidney,other,not reported,male,16703,Day,CPTAC-CCRCC
...,...,...,...,...,...,...,...,...,...
2815,C3L-02888,"Infiltrating duct carcinoma, NOS",Pancreas,other,not reported,female,22949,Day,CPTAC-PDA
2818,C3L-03622,Not Reported,Pancreas,not reported,not reported,female,,Day,CPTAC-PDA
2819,C3L-03629,"Infiltrating duct carcinoma, NOS",Pancreas,other,not reported,male,27489,Day,CPTAC-PDA
2820,C3N-01898,"Infiltrating duct carcinoma, NOS",Pancreas,white,not reported,female,17926,Day,CPTAC-PDA


In [60]:
# Add the 'Available Images' column 
cptac_with_image_availability = add_image_availability(cptac_with_projects, pathology_patients, radiology_patients)

cptac_with_image_availability

,Case ID,Primary Diagnosis,Primary Site,Race,Ethnicity,Sex at Birth,Age at Diagnosis,Age UOM,Project Short Name,Available Images
0,C3N-00721,"Renal cell carcinoma, NOS",Kidney,white,not hispanic or latino,male,,Day,CPTAC-CCRCC,Pathology
1,C3L-02556,"Renal cell carcinoma, NOS",Kidney,white,not hispanic or latino,male,31315,Day,CPTAC-CCRCC,Pathology
2,C3N-01657,"Renal cell carcinoma, NOS",Kidney,white,not reported,male,27232,Day,CPTAC-CCRCC,Pathology
3,C3L-00970,"Renal cell carcinoma, NOS",Kidney,other,not reported,male,20187,Day,CPTAC-CCRCC,Pathology
4,C3L-00991,"Renal cell carcinoma, NOS",Kidney,other,not reported,male,16703,Day,CPTAC-CCRCC,Pathology
...,...,...,...,...,...,...,...,...,...,...
2815,C3L-02888,"Infiltrating duct carcinoma, NOS",Pancreas,other,not reported,female,22949,Day,CPTAC-PDA,Radiology
2818,C3L-03622,Not Reported,Pancreas,not reported,not reported,female,,Day,CPTAC-PDA,Radiology
2819,C3L-03629,"Infiltrating duct carcinoma, NOS",Pancreas,other,not reported,male,27489,Day,CPTAC-PDA,Radiology
2820,C3N-01898,"Infiltrating duct carcinoma, NOS",Pancreas,white,not reported,female,17926,Day,CPTAC-PDA,Radiology


In [61]:
# fix data values to align with TCIA standards 

# Make a copy of the original dataframe
cptac_clinical = cptac_with_image_availability.copy()

# Remove duplicate rows where all values are the same -- patients showing up more than once in PDC data due to multiple samples? not clear
cptac_clinical = cptac_clinical.drop_duplicates()

# Drop all records where 'Project Short Name' contains 'TCGA'
# remove TCGA-OV and TCGA-BRCA cases that are showing up as "CPTAC2 Retrospective" projects -- we'll go with GDC for those
cptac_clinical = cptac_clinical[~cptac_clinical['Project Short Name'].str.contains('TCGA')]

# Correct capitalization and replace 'other' in the 'Race' column
race_corrections = {
    'asian': 'Asian',
    'not reported': 'Not Reported',
    'black or african american': 'Black or African American',
    'american indian or alaska native': 'American Indian or Alaska Native',
    'white': 'White',
    'other': 'Unknown'
}
cptac_clinical['Race'] = cptac_clinical['Race'].replace(race_corrections)

# Correct capitalization in the 'Ethnicity' column
ethnicity_corrections = {
    'not reported': 'Not Reported',
    'hispanic or latino': 'Hispanic or Latino',
    'not hispanic or latino': 'Not Hispanic or Latino'
}
cptac_clinical['Ethnicity'] = cptac_clinical['Ethnicity'].replace(ethnicity_corrections)

# Correct capitalization and replace 'not reported' in the 'Sex at Birth' column
sex_corrections = {
    'male': 'Male',
    'female': 'Female',
    'not reported': 'Unknown'
}
cptac_clinical['Sex at Birth'] = cptac_clinical['Sex at Birth'].replace(sex_corrections)

In [62]:
# Extract unique values from the 'collection' column 
unique_projects = cptac_clinical['Project Short Name'].unique().tolist()

print(unique_projects)

['CPTAC-CCRCC', 'CPTAC-UCEC', 'CPTAC-LSCC', 'CPTAC-GBM', 'CPTAC-PDA', 'CPTAC-LUAD', 'UPENN-GBM', 'CPTAC-OV', 'CPTAC-HNSCC', 'CPTAC-BRCA', 'CPTAC-COAD']


In [63]:
cptac_clinical.to_excel('cptac_clinical_final.xlsx')

# Get TCGA clinical data from GDC


In [64]:
# Check for collections related to TCGA radiology images
# Note: TCIA doesn't host any pathology for TCGA
gdc_collection_list = radiology[radiology['Collection'].str.contains('tcga', case=False, na=False)]['Collection'].unique().tolist()

# add CPTAC datasets (Note: these are referred to as programs in GDC rather than being specific to cancer type like TCIA. We'll resolve this later.)
#gdc_collection_list.extend(['CPTAC-2', 'CPTAC-3'])

print(len(gdc_collection_list))
print(gdc_collection_list)

21
['TCGA-GBM', 'TCGA-BRCA', 'TCGA-LGG', 'TCGA-KIRC', 'TCGA-LUAD', 'TCGA-PRAD', 'TCGA-KIRP', 'TCGA-OV', 'TCGA-LIHC', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-BLCA', 'TCGA-LUSC', 'TCGA-COAD', 'TCGA-THCA', 'TCGA-READ', 'TCGA-UCEC', 'TCGA-ESCA', 'TCGA-STAD', 'TCGA-CESC', 'TCGA-SARC']


In [65]:
cases_endpt = 'https://api.gdc.cancer.gov/cases'

filters = {
    "op": "in",
    "content":{
        "field": "project.project_id",
        "value": gdc_collection_list
        }
    }

fields = [
    "project.project_id",
    "submitter_id",
    ]

fields = ','.join(fields)

expand = [ ## For the allowable values for this list, look under "mapping" at https://api.gdc.cancer.gov/cases/_mapping
    "demographic",
    "diagnoses"
    #"diagnoses.treatments",
    #"exposures",
    #"family_histories"
    ]

expand = ','.join(expand)

params = {
    "filters": json.dumps(filters),
    "expand": expand,
    "fields": fields,
    "format": "TSV", ## This can be "JSON" too
    "size": "20000", ## If you are re-using this for other projects, you may need to modify this and the "from" number.
    "from":"0"
    }

response = requests.get(cases_endpt, params = params)

output = response.content.decode('UTF-8')
clinicalDf = pd.read_csv(io.StringIO(output), sep='\t')

#clinicalDf.to_excel('gdc.xlsx')

/var/folders/19/4p5m1b9j5k9f7qybvc64fbb454krph/T/ipykernel_27283/1455514910.py:40: DtypeWarning: Columns (2,18,19,20,21,39,78,83) have mixed types. Specify dtype option on import or set low_memory=False.
  clinicalDf = pd.read_csv(io.StringIO(output), sep='\t')


In [67]:
# rename subject_id to Case ID
gdc = clinicalDf.rename(columns={'submitter_id': 'Case ID'})

# Add the 'available_images' column to the 'pathology' DataFrame
gdc_with_project_and_image_availability = add_image_availability(gdc, pathology_patients, radiology_patients)

#gdc_with_project_and_image_availability.to_excel('gdc.xlsx')

gdc_with_project_and_image_availability

,demographic.age_at_index,demographic.cause_of_death,demographic.created_datetime,demographic.days_to_birth,demographic.days_to_death,demographic.demographic_id,demographic.ethnicity,demographic.gender,demographic.premature_at_birth,demographic.race,...,diagnoses.0.tumor_grade,diagnoses.0.tumor_regression_grade,diagnoses.0.updated_datetime,diagnoses.0.weiss_assessment_score,diagnoses.0.wilms_tumor_histologic_subtype,diagnoses.0.year_of_diagnosis,id,project.project_id,Case ID,Available Images
0,65.0,NaN,NaN,-24064.0,1484.0,4d9e60db-29bf-55a5-b771-45b205a0536b,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2007.0,85a85a11-7200-4e96-97af-6ba26d680d59,TCGA-OV,TCGA-13-0920,Radiology
1,49.0,NaN,NaN,-17961.0,1447.0,bf1408e4-38fe-591d-b84c-033112b7d068,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2009.0,7922df77-f09a-488c-a1be-58646ceb9b3e,TCGA-OV,TCGA-42-2582,None
2,80.0,NaN,NaN,-29501.0,563.0,6a95eab4-efb0-5637-8480-f4a864f3c478,not reported,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2006.0,8727855e-120a-4216-a803-8cc6cd1159be,TCGA-OV,TCGA-04-1342,None
3,59.0,NaN,NaN,-21635.0,2025.0,55b47e7c-07df-5c84-a4c5-11752b4a80e1,not reported,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2004.0,82e96c6c-a88c-4e52-be56-7f24f6c7b835,TCGA-OV,TCGA-13-1819,None
4,60.0,NaN,NaN,-21963.0,55.0,e6acce8e-ae80-5f1b-846a-24a6d203fe5a,not hispanic or latino,female,NaN,black or african american,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2005.0,ab3dbbbe-eed6-4a35-a505-1815225e86c9,TCGA-OV,TCGA-04-1335,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9577,53.0,NaN,NaN,-19547.0,NaN,eb111509-466e-5511-a767-c881fb9375e6,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2009.0,a798a8cc-4e72-4a8c-9e20-74a14deafd12,TCGA-UCEC,TCGA-BG-A0MS,None
9578,80.0,NaN,NaN,-29260.0,NaN,4bed5f1e-e85b-5763-bb8b-ea0029cf902d,hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2011.0,fe2e89f7-8f4d-420a-a551-4877cf0fd1d3,TCGA-UCEC,TCGA-EO-A3KX,None
9579,83.0,NaN,NaN,-30517.0,456.0,76d2f9cb-ddcb-59c4-9316-1df7c9f3062d,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2008.0,db3a4986-55d5-4ecc-be73-59725dce3c33,TCGA-UCEC,TCGA-EY-A1G8,None
9580,73.0,NaN,NaN,-26992.0,NaN,16d35689-c5a4-50b8-b9ee-af408c742373,not hispanic or latino,female,NaN,black or african american,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2013.0,ff6b5fc8-0572-4b58-b3a5-bcda41badbc8,TCGA-UCEC,TCGA-PG-A914,None


In [68]:
gdc_with_project_and_image_availability = gdc_with_project_and_image_availability[gdc_with_project_and_image_availability['Available Images'] != 'None']

#gdc_with_project_and_image_availability.to_excel('gdc-clinical.xlsx')

gdc_with_project_and_image_availability

,demographic.age_at_index,demographic.cause_of_death,demographic.created_datetime,demographic.days_to_birth,demographic.days_to_death,demographic.demographic_id,demographic.ethnicity,demographic.gender,demographic.premature_at_birth,demographic.race,...,diagnoses.0.tumor_grade,diagnoses.0.tumor_regression_grade,diagnoses.0.updated_datetime,diagnoses.0.weiss_assessment_score,diagnoses.0.wilms_tumor_histologic_subtype,diagnoses.0.year_of_diagnosis,id,project.project_id,Case ID,Available Images
0,65.0,NaN,NaN,-24064.0,1484.0,4d9e60db-29bf-55a5-b771-45b205a0536b,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2007.0,85a85a11-7200-4e96-97af-6ba26d680d59,TCGA-OV,TCGA-13-0920,Radiology
9,59.0,NaN,NaN,-21635.0,1680.0,45f8a427-8d1d-5aae-b1f3-7f698c460b17,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2008.0,8fac8e40-beac-4059-9d54-7ee530598cfd,TCGA-OV,TCGA-13-1408,Radiology
15,50.0,NaN,NaN,-18596.0,NaN,c46f9687-5a61-5a44-b927-5cdc22391434,not hispanic or latino,female,NaN,not reported,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2001.0,0fc8777c-8f12-472f-a0ea-085139b35d1f,TCGA-OV,TCGA-09-1675,Radiology
30,44.0,NaN,NaN,-16222.0,608.0,ec4793f1-4af3-5052-bc55-d865c8d8a000,not reported,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2005.0,a5030259-cf9c-4a58-8710-b9da8ee59320,TCGA-OV,TCGA-10-0937,Radiology
32,59.0,NaN,NaN,-21726.0,2154.0,b6bd947a-dae0-5ef3-905e-02630b777a39,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:23:52.066938-05:00,NaN,NaN,2002.0,a85f6f9c-1e1d-44fc-85eb-3b2d96cfbc61,TCGA-OV,TCGA-13-1488,Radiology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9562,72.0,NaN,NaN,-26638.0,58.0,69387b62-2c68-5b85-a959-7200561ef196,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2008.0,c5df9142-27f6-4d97-933e-c79ea9199362,TCGA-UCEC,TCGA-D1-A15Z,Radiology
9565,48.0,NaN,NaN,-17802.0,NaN,5adae4ab-9d4c-5380-ba44-2fc49810993d,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2009.0,f0b3ab10-43f8-48a8-b460-66ceaaf637db,TCGA-UCEC,TCGA-D1-A175,Radiology
9569,68.0,NaN,NaN,-24886.0,NaN,5e0b5d34-f9df-502e-94e6-7bfeeb92a539,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2008.0,f7010b87-a559-4501-99f9-0fcca1e986e1,TCGA-UCEC,TCGA-D1-A15V,Radiology
9574,82.0,NaN,NaN,-30224.0,NaN,8bd8505e-e3b3-5167-adc2-d6abce781dc6,not hispanic or latino,female,NaN,white,...,Not Reported,NaN,2023-10-06T12:22:33.388084-05:00,NaN,NaN,2008.0,dffc7091-bc27-452c-ad94-1ee214107593,TCGA-UCEC,TCGA-FI-A2CX,Radiology


# in order to align with output from https://tcia-clinical-validator.streamlit.app/
Please create a copy of 'gdc_with_project_and_image_availability' dataframe called 'gdc_tmp' that only contains:
* demographic.ethnicity -> but change header to be Ethnicity
* demographic.gender -> rename to Sex at Birth
* demographic.race -> rename to Race
* diagnoses.0.age_at_diagnosis -> rename to Age at Diagnosis
* diagnoses.0.primary_diagnosis -> Primary Diagnosis
* diagnoses.0.tissue_or_organ_of_origin -> Tissue or Organ of Origin
* project.project_id -> Project Short Name
* create an 'Age UOM' column and populate all values with 'Day'
* Convert the word 'or' in all values that are found in the Tissue or Organ of Origin, Race, Ethnicity columns to be lower case
* set 'Nan' and 'nan' values to be null/NaN values (not strings that say 'Nan' or 'nan')

Side note: there are 40 diagnosis values that don't fall into the GDC CDE?

In [84]:
gdc_tmp = gdc_with_project_and_image_availability[['project.project_id', 'Case ID', 'Available Images', 'diagnoses.0.primary_diagnosis', 'diagnoses.0.tissue_or_organ_of_origin', 'demographic.race', 'demographic.ethnicity', 'demographic.gender', 'diagnoses.0.age_at_diagnosis']].copy()

# Rename columns
gdc_tmp.rename(columns={'demographic.ethnicity': 'Ethnicity',
                         'demographic.gender': 'Sex at Birth',
                         'demographic.race': 'Race',
                         'diagnoses.0.age_at_diagnosis': 'Age at Diagnosis',
                         'diagnoses.0.primary_diagnosis': 'Primary Diagnosis',
                         'diagnoses.0.tissue_or_organ_of_origin': 'Primary Site',
                         'project.project_id': 'Project Short Name'}, inplace=True)

# Convert 'or' to lowercase in specified columns
gdc_tmp['Primary Site'] = gdc_tmp['Primary Site'].str.replace('Or', 'or', case=False)
gdc_tmp['Race'] = gdc_tmp['Race'].str.replace('Or', 'or', case=False)
gdc_tmp['Ethnicity'] = gdc_tmp['Ethnicity'].str.replace('Or', 'or', case=False)

# Create 'Age UOM' column and populate with 'Day'
gdc_tmp['Age UOM'] = 'Day'

# Convert 'Nan' and 'nan' to NaN
gdc_tmp = gdc_tmp.replace(['Nan', 'nan'], pd.NA)

# drop probable GDC data entry issues for 2 subjects in CPTAC-BRCA -- they have no images in TCIA,
#   but the patient ID for 604 aligns with a subject from a different collection
gdc_tmp = gdc_tmp[~gdc_tmp['Case ID'].isin(['604', '1488'])]

# Add the 'Collection' column to the DataFrame to fix CPTAC collections (not just CPTAC-2 and CPTAC-3)
gdc_clinical = add_collection_from_other_dfs(gdc_tmp, pathology, radiology)

# Update the 'Project Short Name' value for the specific 'Case ID' that had overlap between CPTAC-GBM and the CPTAC-GBM CODEX dataset in pathDB
gdc_clinical.loc[gdc_tmp['Case ID'] == 'C3N-03070', 'Project Short Name'] = 'CPTAC-GBM'

In [85]:
# Remove duplicate rows where all values are the same -- patients showing up more than once in PDC data due to multiple samples? not clear
gdc_clinical = gdc_clinical.drop_duplicates()

# Correct capitalization and replace 'other' in the 'Race' column
race_corrections = {
    'asian': 'Asian',
    'not reported': 'Not Reported',
    'black or african american': 'Black or African American',
    'american indian or alaska native': 'American Indian or Alaska Native',
    'native hawaiian or other pacific islander': 'Native Hawaiian or Other Pacific Islander',
    'white': 'White',
    'other': 'Unknown'
}
gdc_clinical['Race'] = gdc_clinical['Race'].replace(race_corrections)
gdc_clinical['Race'] = gdc_clinical['Race'].fillna('Not Reported')

# Correct capitalization in the 'Ethnicity' column
ethnicity_corrections = {
    'not reported': 'Not Reported',
    'hispanic or latino': 'Hispanic or Latino',
    'not hispanic or latino': 'Not Hispanic or Latino'
}
gdc_clinical['Ethnicity'] = gdc_clinical['Ethnicity'].replace(ethnicity_corrections)
gdc_clinical['Ethnicity'] = gdc_clinical['Ethnicity'].fillna('Not Reported')

# Correct capitalization and replace 'not reported' in the 'Sex at Birth' column
sex_corrections = {
    'male': 'Male',
    'female': 'Female',
    'not reported': 'Unknown',
    'Not Reported': 'Unknown'
}
gdc_clinical['Sex at Birth'] = gdc_clinical['Sex at Birth'].replace(sex_corrections)
gdc_clinical['Sex at Birth'] = gdc_clinical['Sex at Birth'].fillna('Unknown')

In [86]:
gdc_clinical.to_excel('gdc-clinical.xlsx')

gdc_clinical

,Project Short Name,Case ID,Available Images,Primary Diagnosis,Primary Site,Race,Ethnicity,Sex at Birth,Age at Diagnosis,Age UOM
0,TCGA-OV,TCGA-13-0920,Radiology,"Serous cystadenocarcinoma, NOS",Ovary,White,Not Hispanic or Latino,Female,24064.0,Day
9,TCGA-OV,TCGA-13-1408,Radiology,"Serous cystadenocarcinoma, NOS",Ovary,White,Not Hispanic or Latino,Female,21635.0,Day
15,TCGA-OV,TCGA-09-1675,Radiology,"Serous cystadenocarcinoma, NOS",Ovary,Not Reported,Not Hispanic or Latino,Female,18596.0,Day
30,TCGA-OV,TCGA-10-0937,Radiology,"Serous cystadenocarcinoma, NOS",Ovary,White,Not Reported,Female,16222.0,Day
32,TCGA-OV,TCGA-13-1488,Radiology,"Serous cystadenocarcinoma, NOS",Ovary,White,Not Hispanic or Latino,Female,21726.0,Day
...,...,...,...,...,...,...,...,...,...,...
9562,TCGA-UCEC,TCGA-D1-A15Z,Radiology,"Endometrioid adenocarcinoma, NOS",Endometrium,White,Not Hispanic or Latino,Female,26638.0,Day
9565,TCGA-UCEC,TCGA-D1-A175,Radiology,"Endometrioid adenocarcinoma, NOS",Endometrium,White,Not Hispanic or Latino,Female,17802.0,Day
9569,TCGA-UCEC,TCGA-D1-A15V,Radiology,"Serous cystadenocarcinoma, NOS",Endometrium,White,Not Hispanic or Latino,Female,24886.0,Day
9574,TCGA-UCEC,TCGA-FI-A2CX,Radiology,"Endometrioid adenocarcinoma, NOS",Endometrium,White,Not Hispanic or Latino,Female,30224.0,Day


In [88]:
# Get the unique values for 'project.project_id' column
unique_project_ids = gdc_clinical['Project Short Name'].unique()

# Display the unique project IDs
print(len(unique_project_ids))
print(unique_project_ids)

21
['TCGA-OV' 'TCGA-BLCA' 'TCGA-COAD' 'TCGA-CESC' 'TCGA-PRAD' 'TCGA-READ'
 'TCGA-SARC' 'TCGA-LUSC' 'TCGA-LIHC' 'TCGA-LUAD' 'TCGA-KIRC' 'TCGA-KIRP'
 'TCGA-LGG' 'TCGA-HNSC' 'TCGA-KICH' 'TCGA-BRCA' 'TCGA-GBM' 'TCGA-ESCA'
 'TCGA-STAD' 'TCGA-THCA' 'TCGA-UCEC']


In [89]:
# View records where 'Project Short Name' is null
null_project_short_name_records = gdc_clinical[gdc_clinical['Project Short Name'].isnull()]

# Display the filtered records
display(null_project_short_name_records)


,Project Short Name,Case ID,Available Images,Primary Diagnosis,Primary Site,Race,Ethnicity,Sex at Birth,Age at Diagnosis,Age UOM


### Note: the UPENN-GBM overlap is valid
Some cases that are marked as CPTAC-GBM in GDC were purposefully put in the UPENN-GBM collection on TCIA per the submitters' request to reduce potential confusion about previously existing radiology images that had been published under UPENN-GBM prior to CPTAC deciding to include these subjects in their genomics/proteomics data.

## create master clinical df to combine CTDC Biobank and GDC (TCGA/CPTAC)

In [101]:
master_clinical = pd.concat([biobank_clinical, cptac_clinical, gdc_clinical], ignore_index=True)

In [102]:
# add primary site where known/obvious

# Create a mapping dictionary
primary_site_mapping = {
    'CMB-AML': 'Blood',
    'CMB-BRCA': 'Breast',
    'CMB-CRC': 'Colon',
    'CMB-GEC': 'Esophagus',
    'CMB-LCA': 'Lung',
    'CMB-OV': 'Ovary',
    'CMB-PCA': 'Prostate',
    'TCGA-LUAD': 'Lung',
    'TCGA-GBM': 'Brain, NOS'
}

# Map 'Project Short Name' to 'Primary Site' using the mapping dictionary
master_clinical['Primary Site'] = master_clinical['Project Short Name'].map(primary_site_mapping).fillna(master_clinical['Primary Site'])

master_clinical


,Project Short Name,Case ID,Available Images,Primary Diagnosis,Race,Ethnicity,Sex at Birth,Age at Enrollment,Age UOM,Primary Site,Age at Diagnosis
0,CMB-MML,MSB-00089,Pathology,Plasma Cell Myeloma,Black or African American,Not Hispanic or Latino,Female,67.0,Year,NaN,NaN
1,CMB-MML,MSB-00140,Radiology; Pathology,Plasma Cell Myeloma,White,Not Hispanic or Latino,Female,65.0,Year,NaN,NaN
2,CMB-MEL,MSB-00205,Pathology,Melanoma,White,Not Hispanic or Latino,Male,63.0,Year,NaN,NaN
3,CMB-CRC,MSB-00241,Pathology,Colorectal Carcinoma,White,Not Hispanic or Latino,Female,59.0,Year,Colon,NaN
4,CMB-MEL,MSB-00263,Radiology; Pathology,Melanoma,White,Not Hispanic or Latino,Male,56.0,Year,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3600,TCGA-UCEC,TCGA-D1-A15Z,Radiology,"Endometrioid adenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,26638.0
3601,TCGA-UCEC,TCGA-D1-A175,Radiology,"Endometrioid adenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,17802.0
3602,TCGA-UCEC,TCGA-D1-A15V,Radiology,"Serous cystadenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,24886.0
3603,TCGA-UCEC,TCGA-FI-A2CX,Radiology,"Endometrioid adenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,30224.0


In [103]:
master_clinical.to_excel('crdc-clinical.xlsx', index = False)

In [58]:
# Extract unique values from the specified columns
unique_race = master_clinical['Race'].unique().tolist()
unique_ethnicity = master_clinical['Ethnicity'].unique().tolist()
unique_sex_at_birth = master_clinical['Sex at Birth'].unique().tolist()

print("Unique Race values:", unique_race)
print("Unique Ethnicity values:", unique_ethnicity)
print("Unique Sex at Birth values:", unique_sex_at_birth)

Unique Race values: ['Black or African American', 'White', 'Not Reported', 'Asian', 'Unknown', 'American Indian or Alaska Native', 'Native Hawaiian or other Pacific Islander', 'Native Hawaiian or Other Pacific Islander']
Unique Ethnicity values: ['Not Hispanic or Latino', 'Hispanic or Latino', 'Not Reported', 'Unknown']
Unique Sex at Birth values: ['Female', 'Male', 'Unknown']


In [59]:
master_clinical

,Project Short Name,Case ID,Available Images,Primary Diagnosis,Race,Ethnicity,Sex at Birth,Age at Enrollment,Age UOM,Primary Site,Age at Diagnosis
0,CMB-MML,MSB-00089,Pathology,Plasma Cell Myeloma,Black or African American,Not Hispanic or Latino,Female,67.0,Year,NaN,NaN
1,CMB-MML,MSB-00140,Both,Plasma Cell Myeloma,White,Not Hispanic or Latino,Female,65.0,Year,NaN,NaN
2,CMB-MEL,MSB-00205,Pathology,Melanoma,White,Not Hispanic or Latino,Male,63.0,Year,NaN,NaN
3,CMB-CRC,MSB-00241,Pathology,Colorectal Carcinoma,White,Not Hispanic or Latino,Female,59.0,Year,NaN,NaN
4,CMB-MEL,MSB-00263,Both,Melanoma,White,Not Hispanic or Latino,Male,56.0,Year,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3683,TCGA-UCEC,TCGA-D1-A15Z,Radiology,"Endometrioid adenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,26638.0
3684,TCGA-UCEC,TCGA-D1-A175,Radiology,"Endometrioid adenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,17802.0
3685,TCGA-UCEC,TCGA-D1-A15V,Radiology,"Serous cystadenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,24886.0
3686,TCGA-UCEC,TCGA-FI-A2CX,Radiology,"Endometrioid adenocarcinoma, NOS",White,Not Hispanic or Latino,Female,NaN,Day,Endometrium,30224.0


In [61]:
master_clinical.to_excel('nci_program_clinical.xlsx')